In [1]:
# using Pkg; Pkg.add("Distributions"); Pkg.add("OnlineStats")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
 Installed URIParser ──────── v0.4.0
 Installed Rmath ──────────── v0.5.0
 Installed StatsFuns ──────── v0.8.0
 Installed PDMats ─────────── v0.9.9
 Installed QuadGK ─────────── v2.0.3
 Installed SpecialFunctions ─ v0.7.2
 Installed BinDeps ────────── v0.8.10
 Installed Distributions ──── v0.21.1
 Installed Arpack ─────────── v0.3.1
  Updating `~/.julia/environments/v1.2/Project.toml`
  [31c24e10] + Distributions v0.21.1
  Updating `~/.julia/environments/v1.2/Manifest.toml`
  [7d9fca2a] + Arpack v0.3.1
  [9e28174c] + BinDeps v0.8.10
  [31c24e10] + Distributions v0.21.1
  [90014a1f] + PDMats v0.9.9
  [1fd47b50] + QuadGK v2.0.3
  [79098fc4] + Rmath v0.5.0
  [276daf66] + SpecialFunctions v0.7.2
  [4c63d2b9] + StatsFuns v0.8.0
  [30578b45] + URIParser v0.4.0
  [4607b0f0] + SuiteSparse 
  Building Rmath ───────────→ `~/.julia/packages/Rmath

In [2]:
using Distributions, OnlineStats

function queue1(until::Real, burnin::Real,
                ad::Distribution, sd::Distribution)
    now, nextArrival, nextDeparture = 0.0, rand(ad), Inf
    queue, waits = Float64[], Mean()
    while now < until
        if nextArrival < nextDeparture
            now = nextArrival
            if isempty(queue)
                nextDeparture = nextArrival + rand(sd)
            end
            push!(queue, nextArrival)
            nextArrival += rand(ad)
        else
            now = nextDeparture
            insystem = nextDeparture - popfirst!(queue)
            burnin < now < until && fit!(waits, insystem)
            nextDeparture += isempty(queue) ? Inf : rand(sd)
        end 
    end
    value(waits)
end

mm1_exact(ad::Exponential, sd::Exponential) =
    1/(1/mean(sd)-1/mean(ad))

function bootCI(data, stat::Function, CI::Float64, reps::Integer)
    boot = [stat(rand(data, length(data))) for i in 1:reps]
    low, high = quantile(boot, [(1-CI)/2, (1+CI)/2])
    (value=stat(data), low=low, high=high)
end

┌ Info: Precompiling Distributions [31c24e10-a181-5473-b8eb-7969acd0382f]
└ @ Base loading.jl:1242


bootCI (generic function with 1 method)

In [3]:
ad = Exponential(1.3)
sd = Exponential(0.95)

Exponential{Float64}(θ=0.95)

In [4]:
@time res = [queue1(2^14, 2^12, ad, sd) for i in 1:2^12];

  2.045617 seconds (487.40 k allocations: 29.459 MiB, 0.47% gc time)


In [5]:
exact = mm1_exact(ad, sd)

3.5285714285714285

In [6]:
@time println(bootCI(res .- exact, mean, 0.99, 2^14))

(value = -0.0017247093297056342, low = -0.011037472009682005, high = 0.007887692853284327)
  0.928032 seconds (1.18 M allocations: 572.150 MiB, 4.27% gc time)


In [7]:
function bootCI(data, stat::Function, CI::Float64, reps::Integer)
    tmp = similar(data)
    boot = [stat(rand!(tmp, data)) for i in 1:reps]
    low, high = quantile(boot, [(1-CI)/2, (1+CI)/2])
    (value=stat(data), low=low, high=high)
end

bootCI (generic function with 1 method)

In [8]:
Pkg.add("StatsBase")

 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]


In [9]:
using StatsBase, Distributions
function bayesbootCI(data, stat, CI::Float64, reps::Integer)
    d = Dirichlet(length(data), 1)
    boot = [stat(data, weights(rand(d))) for i in 1:reps]
    low, high = quantile(boot, [(1-CI)/2, (1+CI)/2])
    (value=stat(data), low=low, high=high)
end

bayesbootCI (generic function with 1 method)

In [10]:
@time println(bayesbootCI(res .- exact, mean, 0.99, 2^14))

(value = -0.0017247093297056342, low = -0.01140267598516432, high = 0.0077771160478110005)
  1.007242 seconds (495.10 k allocations: 537.832 MiB, 3.23% gc time)
